In [1]:
import salabim as sim
import random

In [14]:
# Parameters
# I USED TIME IN MINUTES
number_of_arrivals = [30,80,110,90,80,70,80,90,100,120,90,40] # Number of customers arriving at the shop each hour
number_of_checkouts = 3
chance_taking_cart = 80 # There is a chance of 80% someone takes a cart instead of a basket
number_of_carts = 45 #There are only 45 carts available     
time_per_department = 5 # Time it takes in department A, B, E, F, G, THIS NEEDS TO BE CHANGED AND DEPENDS ON THE NUMBER OF PRODUCTS
product_time = random.uniform(1/3,1/2) # Time it takes to pick up 1 product and move to the next one
bread_employees = 4 # Number of employees at the counter of the bread department
bread_time = 2 # Average time at the bread counter
cheese_employees = 3 # Number of employees at the counter of the cheese department
cheese_time = 1 # Average time at the cheese counter
products_per_department = 5 # THIS NEEDS TO BE CHANGED, this should be the number of products a customer picks at each department
checkout_product_time = 1.1/60  #Time at the checkout it takes to scan 1 product
checkout_pay_time = random.uniform(0.66, 1) # Time it takes at the checkout to pay
normal_route = 40 # Only 40% takes the route a-b-c-d-e-f-g, other customers take b-c-d-e-a-f-g as route 

# This calculates the number of people arriving at the shop each hour, by looking at the list and dividing that number by 60
def arrivals_per_hour(hour):
    number_of_customers = number_of_arrivals[hour % len(number_of_arrivals)] 
    if number_of_customers > 0:
        return 60 / number_of_customers
    else:
        return 0

# Customer class
class Customer(sim.Component):
    def process(self):
        arrival_time = env.now() #Set arrival time when the customer arrives at the shop
        
        #Deciding on shopping cart or basket:
        if random.randint(0,100) < chance_taking_cart:
            #with shopping_cart.request() as request:
            self.request(shopping_cart)
            transport_type = "shopping_cart"
            waiting_cart_time = env.now() - arrival_time
            print("Minutes waited for a shopping cart: ", waiting_cart_time)
            start_shopping_time = env.now()
            
        else:
            #with basket.request() as request:
            self.request(basket)
            transport_type = "basket"
            waiting_cart_time = env.now() - arrival_time
            print("Minutes waited for a shopping basket: ", waiting_cart_time)
            start_shopping_time = env.now()
                
        #TO DO: IMPLEMENT THE MAX NUMBER OF CARTS == 45
            
        # Deciding route
        total_products = 0
        if random.randint(0,100) > normal_route:
            route = ['B', 'C', 'D', 'E', 'A', 'F', 'G'] 
        else:
            route = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] 
        
        # Walking past each department    
        for department in route:
            number_of_products = 5 #THIS NEEDS TO BE CHANGED, and depends on the department
            total_products += number_of_products
            
            #CHEESE AND BREAD DEPARTMENT COUNTER TIME NEEDS TO BE ADDED
            self.hold(random.expovariate(1/time_per_department)) # This is the time the customer is spending at the department
        
        #calculate the time the customer was shopping
        time_spend_shopping = env.now() - start_shopping_time        
        print("Minutes spend in the shop: ", time_spend_shopping)
        
        # Time to check out
        start_checkout = env.now() #Set start time of the checkout
        self.request(checkout)
        
        # Calculate waiting time for a checkout
        waiting_checkout_time = env.now() - start_checkout
        print(f"Customer starts checkout after waiting for {waiting_checkout_time:.2f} minutes.")
        
        # Calculate actual time someone is helped at the checkout
        checkout_time = total_products * checkout_product_time + checkout_pay_time
        self.hold(checkout_time)
        actual_checkout_time = env.now() - start_checkout
        print(f"Payment is done after {actual_checkout_time:.2f} minutes.")
        
        # Putting the shopping cart or basket back
        if transport_type == "basket":
            basket.release()
        else:
            shopping_cart.release()
                
# Creating the customers
class CustomerGenerator(sim.Component):
    def process(self):
        while True:
            current_timestep = int(env.now()//60)
            # Decide the number of customers of this hour by calling the function at the top of this cell
            new_arrivals = arrivals_per_hour(current_timestep)
            self.hold(random.expovariate(1/new_arrivals))
            Customer()

# Set environment
env = sim.Environment()

# Define resources
checkout = sim.Resource(name="checkout", capacity=number_of_checkouts)
shopping_cart = sim.Resource(name="shopping_cart", capacity=number_of_carts)
basket = sim.Resource(name="basket", capacity = 300)

# Create the customers
CustomerGenerator()

# Start simulation
env.run(till=500)  


Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping basket:  0.0
Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping basket:  0.0
Minutes waited for a shopping cart:  0.0
Minutes waited for a shopping cart:  0.0
Minutes spend in the shop:  20.686661999876957
Customer starts checkout after waiting for 0.00 minutes.
Payment is done after 1.34 minutes.
Minutes spend in the shop:  24.989585536823185
Customer starts checkout after waiting for 0.00 minutes.
Minutes waited for a shopping cart:  0.0
Payment is done after 1.34 minutes.
Minutes spend in the shop:  10.503685939912963
Customer starts checkout after waiting for 0.00 minutes.
Payment is done after 1.34 minutes.
Minutes spend in the shop:  16.06413475425872
Customer starts checkout after waiting